This is the code I used to filter out the relevant columns for the VA educational attainment data. I saved a new csv file with the selected data, so nobody really needs to run this code again unless we change our mind about what data we want to capture from the initial sheet. 

-Mariska (2/14/22)

In [2]:
import pandas as pd

#import the original data set
ed_data = pd.read_csv("EAG_Educational_Attainment_by_Locality.csv")

In [3]:
#view the original data set
ed_data.head()

,Locality,locality_fips,state_name,state_fips,rural_or_non_rural_flag,pct_no_high_school_diploma_loc,cnt_no_high_school_diploma_loc,pct_high_school_grad_loc,cnt_total_high_school_grad_loc,pct_total_high_school_grad_loc,...,pct_high_school_grad_Hispanic_loc,pct_bachelors_degree_Caucasian_loc,pct_bachelors_degree_Black_loc,pct_bachelors_degree_Asian_loc,pct_bachelors_degree_Hispanic_loc,pct_poverty_education_no_high_school_loc,pct_poverty_education_high_school_loc,pct_poverty_education_bachelors_or_higher_loc,pct_pursuing_HighEd_All,cnt_pursuing_HighEd_All
0,Surry,51181,Virginia,51,Non-Rural,0.154303,832,0.371662,4560,0.845697,...,0.878,0.225,0.177,0.000,0.293,0.245,0.150,0.052,0.66,39
1,Clarke,51043,Virginia,51,Non-Rural,0.106878,1223,0.297387,10220,0.893122,...,0.751,0.333,0.278,0.149,0.155,0.161,0.102,0.025,0.75,131
2,Mecklenburg,51117,Virginia,51,Rural,0.180888,4507,0.328303,20409,0.819112,...,0.697,0.256,0.107,0.130,0.122,0.322,0.185,0.070,0.65,186
3,Augusta,51015,Virginia,51,Rural,0.129816,7881,0.392281,52828,0.870184,...,0.571,0.236,0.053,0.707,0.114,0.163,0.092,0.030,0.58,413
4,Goochland,51075,Virginia,51,Non-Rural,0.095134,1789,0.242382,17016,0.904866,...,0.830,0.474,0.148,0.761,0.308,0.105,0.083,0.026,0.66,142


In [8]:
#create a new dataframe with selected columns

ed_new = ed_data[['Locality','locality_fips','rural_or_non_rural_flag', 'pct_no_high_school_diploma_loc', 'pct_high_school_grad_loc','pct_Some_College_Associates_loc','pct_bachelors_degree_+_loc']]
ed_new.head(3)

,Locality,locality_fips,rural_or_non_rural_flag,pct_no_high_school_diploma_loc,pct_high_school_grad_loc,pct_Some_College_Associates_loc,pct_bachelors_degree_+_loc
0,Surry,51181,Non-Rural,0.154303,0.371662,0.276335,0.197700
1,Clarke,51043,Non-Rural,0.106878,0.297387,0.296426,0.299310
2,Mecklenburg,51117,Rural,0.180888,0.328303,0.304704,0.186105


In [9]:
#export the new data frame as a CSV file
ed_new.to_csv('VA_ed_data_cleaned.csv')

In [11]:
#check new file
data = pd.read_csv('VA_ed_data_cleaned.csv')
data.head()

,Unnamed: 0,Locality,locality_fips,rural_or_non_rural_flag,pct_no_high_school_diploma_loc,pct_high_school_grad_loc,pct_Some_College_Associates_loc,pct_bachelors_degree_+_loc
0,0,Surry,51181,Non-Rural,0.154303,0.371662,0.276335,0.197700
1,1,Clarke,51043,Non-Rural,0.106878,0.297387,0.296426,0.299310
2,2,Mecklenburg,51117,Rural,0.180888,0.328303,0.304704,0.186105
3,3,Augusta,51015,Rural,0.129816,0.392281,0.260159,0.217744
4,4,Goochland,51075,Non-Rural,0.095134,0.242382,0.267854,0.394629
